In [ ]:
%pip install metagpt -q

In [8]:
import nest_asyncio
nest_asyncio.apply()

## 单一智能体

- 定义动作

在 MetaGPT 中，类 Action 是动作的逻辑抽象。用户可以通过简单地调用 self._aask 函数令 LLM 赋予这个动作能力，即这个函数将在底层调用 LLM api。

在我们的场景中，我们定义了一个 SimpleWriteCode 子类 Action。虽然它主要是一个围绕提示和 LLM 调用的包装器，但我们认为这个 Action 抽象更直观。在下游和高级任务中，使用它作为一个整体感觉更自然，而不是分别制作提示和调用 LLM，尤其是在智能体的框架内。

In [51]:
from metagpt.actions import Action
import re

class SimpleWriteCode(Action):
    PROMPT_TEMPLATE: str = """
    Write a python function that can {instruction} and provide two runnnable test cases.
    Return ```python your_code_here ``` with NO other texts,
    your code:
    """

    name: str = "SimpleWriteCode"

    async def run(self, instruction: str):
        prompt = self.PROMPT_TEMPLATE.format(instruction=instruction)

        rsp = await self._aask(prompt)

        code_text = SimpleWriteCode.parse_code(rsp)

        return code_text

    @staticmethod
    def parse_code(rsp):
        pattern = r"```python(.*)```"
        match = re.search(pattern, rsp, re.DOTALL)
        code_text = match.group(1) if match else rsp
        return code_text

- 定义角色

在 MetaGPT 中，Role 类是智能体的逻辑抽象。一个 Role 能执行特定的 Action，拥有记忆、思考并采用各种策略行动。基本上，它充当一个将所有这些组件联系在一起的凝聚实体。目前，让我们只关注一个执行动作的智能体，并看看如何定义一个最简单的 Role。

在这个示例中，我们创建了一个 SimpleCoder，它能够根据人类的自然语言描述编写代码。步骤如下：

    我们为其指定一个名称和配置文件。
    我们使用 self._init_action 函数为其配备期望的动作 SimpleWriteCode。
    我们覆盖 _act 函数，其中包含智能体具体行动逻辑。我们写入，我们的智能体将从最新的记忆中获取人类指令，运行配备的动作，MetaGPT将其作为待办事项 (self.rc.todo) 在幕后处理，最后返回一个完整的消息。

In [52]:
from metagpt.roles import Role
from metagpt.actions import Action
from metagpt.logs import logger
from metagpt.roles.role import Role, RoleReactMode
from metagpt.schema import Message

class SimpleCoder(Role):
    name: str = "Alice"
    profile: str = "SimpleCoder"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([SimpleWriteCode])

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: to do {self.rc.todo}({self.rc.todo.name})")
        todo = self.rc.todo  # todo will be SimpleWriteCode()

        msg = self.get_memories(k=1)[0]  # find the most recent messages
        code_text = await todo.run(msg.content)
        msg = Message(content=code_text, role=self.profile, cause_by=type(todo))

        return msg

In [14]:
import asyncio

from metagpt.context import Context

async def main():
    msg = "write a function that calculates the sum of a list"
    context = Context()
    role = SimpleCoder(context=context)
    logger.info(msg)
    result = await role.run(msg)
    logger.info(result)

await main()

2024-05-11 14:38:07.432 | INFO     | __main__:main:9 - write a function that calculates the sum of a list
2024-05-11 14:38:07.433 | INFO     | __main__:_act:16 - Alice(SimpleCoder): to do SimpleWriteCode(SimpleWriteCode)


```python
def calculate_sum(lst):
    return sum(lst)

# Test cases
print(calculate_sum([1, 2, 3]))  # Output should be 6
print(calculate_sum([4, 5, 6]))  # Output should be 15
```


2024-05-11 14:38:09.002 | INFO     | metagpt.utils.cost_manager:update_cost:57 - Total running cost: $0.000 | Max budget: $10.000 | Current cost: $0.000, prompt_tokens: 70, completion_tokens: 60
2024-05-11 14:38:09.003 | INFO     | __main__:main:11 - SimpleCoder: 
def calculate_sum(lst):
    return sum(lst)

# Test cases
print(calculate_sum([1, 2, 3]))  # Output should be 6
print(calculate_sum([4, 5, 6]))  # Output should be 15



## 多智能体入门

在上一章中，我们简要讨论了单智能体的创建。虽然对许多情况来说，单智能体可能已经足够，但更复杂的任务通常需要协作和团队合作，这也就是多智能体为什么必不可少的原因。MetaGPT的核心优势也在于轻松灵活地开发一个智能体团队。在MetaGPT框架下，用户可以通过少量代码实现智能体之间的交互。

完成本节，你将能够：

    理解智能体之间如何进行交互
    开发你的第一个智能体团队

In [53]:
from metagpt.actions import Action, UserRequirement
from metagpt.logs import logger
from metagpt.roles import Role
from metagpt.schema import Message
from metagpt.team import Team

In [54]:
def parse_code(rsp):
    pattern = r"```python(.*)```"
    match = re.search(pattern, rsp, re.DOTALL)
    code_text = match.group(1) if match else rsp
    return code_text

In [55]:
class SimpleWriteCode(Action):
    PROMPT_TEMPLATE: str = """
    Write a python function that can {instruction}.
    Return ```python your_code_here ``` with NO other texts,
    your code:
    """
    name: str = "SimpleWriteCode"

    async def run(self, instruction: str):
        prompt = self.PROMPT_TEMPLATE.format(instruction=instruction)

        rsp = await self._aask(prompt)

        code_text = parse_code(rsp)

        return code_text

In [56]:
class SimpleWriteTest(Action):
    PROMPT_TEMPLATE: str = """
    Context: {context}
    Write {k} unit tests using pytest for the given function, assuming you have imported it.
    Return ```python your_code_here ``` with NO other texts,
    your code:
    """

    name: str = "SimpleWriteTest"

    async def run(self, context: str, k: int = 3):
        prompt = self.PROMPT_TEMPLATE.format(context=context, k=k)

        rsp = await self._aask(prompt)

        code_text = parse_code(rsp)

        return code_text

In [57]:
class SimpleWriteReview(Action):
    PROMPT_TEMPLATE: str = """
    Context: {context}
    Review the test cases and provide one critical comments:
    """

    name: str = "SimpleWriteReview"

    async def run(self, context: str):
        prompt = self.PROMPT_TEMPLATE.format(context=context)

        rsp = await self._aask(prompt)

        return rsp

在许多多智能体场景中，定义Role可能只需几行代码。对于SimpleCoder，我们做了两件事：

    使用 set_actions 为Role配备适当的 Action，这与设置单智能体相同
    多智能体操作逻辑：我们使Role _watch 来自用户或其他智能体的重要上游消息。回想我们的SOP，SimpleCoder接收用户指令，这是由MetaGPT中的UserRequirement引起的Message。因此，我们添加了 self._watch([UserRequirement])。

这就是用户需要做的全部。对于那些对底层机制感兴趣的人，请参见本教程的本章中的机制解释。

In [58]:
class SimpleCoder(Role):
    name: str = "Alice"
    profile: str = "SimpleCoder"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._watch([UserRequirement])
        self.set_actions([SimpleWriteCode])

In [71]:
class SimpleTester(Role):
    name: str = "Bob"
    profile: str = "SimpleTester"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([SimpleWriteTest])
        self._watch([SimpleWriteCode, SimpleWriteReview])  # feel free to try this too

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: to do {self.rc.todo}({self.rc.todo.name})")
        todo = self.rc.todo

        # context = self.get_memories(k=1)[0].content # use the most recent memory as context
        context = self.get_memories()  # use all memories as context

        code_text = await todo.run(context, k=5)  # specify arguments
        msg = Message(content=code_text, role=self.profile, cause_by=type(todo))

        return msg

In [72]:
class SimpleReviewer(Role):
    name: str = "Charlie"
    profile: str = "SimpleReviewer"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([SimpleWriteReview])
        self._watch([SimpleWriteTest])

## 创建一个团队并添加角色

现在我们已经定义了三个 Role，是时候将它们放在一起了。我们初始化所有角色，设置一个 Team，并hire 它们。

运行 Team，我们应该会看到它们之间的协作！

In [75]:
import asyncio
import typer
from metagpt.logs import logger
from metagpt.team import Team
app = typer.Typer()
from pydantic import Field

@app.command()
async def main( 
    idea: str,
    investment:float = 3.0 , # Field(default=3.0, description="Dollar amount to invest in the AI company."),
    n_round: int = 8 #int = Field(default=5, description="Number of rounds for the simulation."),
):
    logger.info(idea)

    team = Team()
    team.hire(
        [
            SimpleCoder(),
            SimpleTester(),
            SimpleReviewer(),
        ]
    )

    team.invest(investment=investment)
    team.run_project(idea)
    await team.run(n_round=n_round)

In [76]:
await main(idea='write a function that calculates the product of a list')

2024-05-11 15:01:28.770 | INFO     | __main__:main:14 - write a function that calculates the product of a list
2024-05-11 15:01:28.801 | INFO     | metagpt.team:invest:90 - Investment: $3.0.
2024-05-11 15:01:28.803 | INFO     | metagpt.roles.role:_act:391 - Alice(SimpleCoder): to do SimpleWriteCode(SimpleWriteCode)


```python
def calculate_product(lst):
    product = 1
    for num in lst:
        product *= num

2024-05-11 15:01:29.672 | INFO     | metagpt.utils.cost_manager:update_cost:57 - Total running cost: $0.000 | Max budget: $3.000 | Current cost: $0.000, prompt_tokens: 65, completion_tokens: 30
2024-05-11 15:01:29.674 | INFO     | __main__:_act:12 - Bob(SimpleTester): to do SimpleWriteTest(SimpleWriteTest)



    return product
```
```python
import pytest

def test_empty_list():
    assert calculate_product([]) == 1

def test_single_element():
    assert calculate_product([5]) == 5

def test_positive_numbers():
    assert calculate_product([2, 3, 4]) == 24

def test_negative_numbers():
    assert calculate_product([-2, -3, -4]) == -24

def test_mixed_numbers():
    assert calculate_product([2, -3, 4])

2024-05-11 15:01:31.869 | INFO     | metagpt.utils.cost_manager:update_cost:57 - Total running cost: $0.001 | Max budget: $3.000 | Current cost: $0.000, prompt_tokens: 111, completion_tokens: 105
2024-05-11 15:01:31.871 | INFO     | metagpt.roles.role:_act:391 - Charlie(SimpleReviewer): to do SimpleWriteReview(SimpleWriteReview)


...

, 10]) == 3628800
```  
As a SimpleReviewer, one critical comment on the test cases provided is that there is no test case for handling floating-point numbers. Including a test case with floating-point numbers would be beneficial to ensure that the function can handle decimal values correctly and does not encounter any precision issues during the calculation

2024-05-11 15:01:42.914 | INFO     | metagpt.utils.cost_manager:update_cost:57 - Total running cost: $0.005 | Max budget: $3.000 | Current cost: $0.001, prompt_tokens: 694, completion_tokens: 61
2024-05-11 15:01:42.916 | INFO     | __main__:_act:12 - Bob(SimpleTester): to do SimpleWriteTest(SimpleWriteTest)


 of the product.
```python
import pytest

def test_empty_list():
    assert calculate_product([]) == 1

def test_single_element():
    assert calculate_product([5]) == 5

def test_positive_numbers():
    assert calculate_product([2, 3, 4]) == 24

def test_negative_numbers():
    assert calculate_product([-2, -3, -4]) == -24

def test_mixed_numbers():
    assert calculate_product([2, -3,

2024-05-11 15:01:44.750 | INFO     | metagpt.utils.cost_manager:update_cost:57 - Total running cost: $0.006 | Max budget: $3.000 | Current cost: $0.001, prompt_tokens: 784, completion_tokens: 105


 4]) == -24
```
